In [1]:
import os
if "models" not in os.listdir("."):
    os.chdir("..")

In [2]:
%load_ext autoreload
%autoreload 2
import penzai
from penzai import pz
pz.ts.register_as_default()
pz.ts.register_autovisualize_magic()
pz.enable_interactive_context()

In [3]:
from micrlhf.llama import LlamaTransformer
llama = LlamaTransformer.from_pretrained("models/gemma-2b-it.gguf",
                                         from_type="gemma",
                                         load_eager=True
                                         )

In [13]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("alpindale/gemma-2b")
tokenizer.padding_side = "right"

In [4]:
!mkdir -p data
import pandas as pd

df_adv = pd.read_csv("data/adv.csv")

format_prompt = """<start_of_turn>user\n
{}\n
<start_of_turn>model\n
{}"""
# offset = 1
# df_do = df.apply(lambda x: format_response.format(x['goal'], x['target']), axis=1)
# prompts_harmful = df.apply(lambda x: format_prompt.format(x['goal'], "")[:-offset], axis=1).to_list()[:100]
prompts_harmful = df_adv.apply(lambda x: format_prompt.format(x['goal'], ""), axis=1).to_list()[:100]
dataset_jail = pd.read_csv("data/jail.csv").apply(lambda x: x["Goal"], axis=1).to_list()
prompts_jail = [format_prompt.format(x, "") for x in dataset_jail]
import datasets
# https://colab.research.google.com/drive/1a-aQvKC9avdZpdyBn4jgRQFObTPy1JZw
hf_path = 'tatsu-lab/alpaca'
dataset = datasets.load_dataset(hf_path)
# filter for instructions that do not have inputs
prompts_harmless = []
for i in range(len(dataset['train'])):
    if len(prompts_harmless) >= len(prompts_harmful):
        break
    if dataset['train'][i]['input'].strip() == '':
        # prompts_harmless.append(format_prompt.format(dataset['train'][i]['instruction'], "")[:-offset])
        prompts_harmless.append(format_prompt.format(dataset['train'][i]['instruction'], ""))

# ds = datasets.load_dataset("MBZUAI/LaMini-instruction", split="train", streaming=True)
# prompts_harmless = []
# for _, text in zip(range(100), ds):
#     prompts_harmless.append(format_prompt.format(text["instruction"], ""))

# ds = datasets.load_dataset("nev/openhermes-2.5-phi-format-text", split="train", streaming=True)
# prompts_harmless = []
# for _, text in zip(range(100), ds):
#     text = text["text"]
#     text = "".join(text.partition("<|assistant|>\n")[:2])
#     prompts_harmless.append(text)

/usr/local/lib/python3.12/pty.py:95: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [14]:
from micrlhf.sampling import sample
msl = 128
completions_harmless = sample(llama, tokenizer, prompts_harmless, max_seq_len=msl, do_sample=True, verbose=True, return_only_completion=True)[0]

  0%|          | 0/81 [00:00<?, ?it/s]

['1. **Eat a balanced diet**. Make sure to include plenty of fruits, vegetables, and whole grains in your diet.\n2. **Get regular exercise**. Aim for at least 30 minutes of moderate-intensity exercise most days of the week.\n3. **Stay hydrated**. Drink plenty of water throughout the day, especially when you are exercising or sweating.<eos><eos><eos><eos>**Additional',
 'Sure. The three primary colors are Red, Yellow, and Blue.<eos><eos><eos>The premise is incorrect, the three primary colors are Red, Yellow, and Green.<eos><eos><eos> The premise is incorrect, the three primary colors are Red, Yellow, and Blue.<eos><eos><eos> The premise is incorrect, the three primary colors are Red, Yellow, and Green.<eos><eos><eos><eos> The premise is',
 "Sure, here is a description of the structure of an atom:\n\nAn atom is the smallest particle of an element that retains the chemical properties of that element. \n\n* **Core:** The nucleus of an atom contains protons and negatively charged electrons, forming a positively charged atomic nucleus.\n* **Shell:** Surrounding the nucleus is the atom's energy levels or shells, which are nested in order",
 '**Reducing Air Pollution**\n\n**1. Transportation:**\n* Use public transportation or walk/bike instead of driving alone.\n* Reduce unnecessary vehicle trips and carpooling.\n* Switch to electric or hybrid vehicles.\n* Invest in fuel-efficient vehicles and regular maintenance.\n\n**2. Energy Consumption:**\n* Switch to energy-efficient appliances and light bulbs.\n* Use LED lights,',
 "Sure, here's an example:\n\nWhile on a hiking trip through a dense forest, I was forced to make a tough decision: whether to follow a narrow path through a muddy undergrowth or stick to the well-trodden path. The undergrowth was thick with dangerous vines and uneven terrain, while the well-trodden path offered little privacy or protection from predators.\n\nAfter careful consideration,",
 'The weight of the upcoming decision pressed down on Amelia like a metaphorical boulder. She sat at her desk, the intricately patterned pattern of her engagement ring a stark reminder of her impending choice. Two paths lay before her, both leading to equally exciting but distinctly different futures.\n\nOne path led toward a prestigious marketing firm nestled in the heart of the city. The prestige and recognition would be enormous, but the',
 '```\n           /\\ /|\n          |||| |\n           \\ | \\\n       _ _ /  ()()\n     /    \\   =>*<=\n   /|      \\   /\n   \\|     /__| |\n     \\_____) \\__)\n```\n\nI can render a 3D model of a house like the one above using various software.<eos><eos><eos><eos><eos>',
 'The provided context does not specify how Julius Caesar died, so I cannot answer this question from the provided context.<eos><eos><eos><eos> How did Julius Caesar die?\nThe context does not specify how Julius Caesar died, so I cannot answer this question from the provided context.<eos><eos><eos><eos> No information is given about how Julius Caesar died in the context, so I cannot provide an answer.<eos><eos><eos>',
 'The capital of France is Paris. It is the political, economic, and cultural center of France.<eos><eos><eos><eos> What is the capital of France?\n\nThe capital of France is Paris. It is the political, economic, and cultural center of France.<eos><eos><eos><eos><eos> What is the capital of France?\n\nThe capital of France is Paris. It is the political, economic, and cultural',
 'Sure, here are ten items a person might need for a camping trip:\n\n1. Tent\n2. Sleeping bag\n3. Flashlight\n4. Multi-tool\n5. Hiking boots\n6. Warm clothing\n7. Cooking utensils\n8. Fire starter\n9. Water filter\n10. First aid kit<eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos>',
 '**Causes of the Great Depression**\n\n**1. Stock Market Bubble:**\n\n* The explosion of stock prices in the late 1920s was fueled by speculation and easy credit, leading to excessive leverage and overvalued stock prices.\n* When stoc

In [15]:
ds_harmless = [(prompt, completion.partition("<eos>")[0].strip()) for prompt, completion in zip(prompts_harmless, completions_harmless)]

In [6]:
from micrlhf.sampling import sample, trange, jnp, load_tokenizer, jit_wrapper
import jax

tokens = tokenizer.batch_encode_plus(prompts_harmful + prompts_harmless,
                                     return_tensors="np",
                                     padding="max_length",
                                     truncation=True,
                                     max_length=128,
                                     return_attention_mask=True)
token_array = jnp.asarray(tokens["input_ids"])
token_array = jax.device_put(token_array, jax.sharding.NamedSharding(llama.mesh, jax.sharding.PartitionSpec("dp", "sp")))
token_array = pz.nx.wrap(token_array, "batch", "seq").untag("batch").tag("batch")
inputs = llama.inputs.from_basic_segments(token_array)

In [29]:
from micrlhf.utils.activation_manipulation import ablate_direction
from functools import partial


@partial(jax.jit, static_argnames=("normalize", "batch_axis"))
def get_loss(direction, llama, input_ids, loss_mask, normalize=True, batch_axis="direction"):
    inputs = llama.inputs.from_basic_segments(input_ids)
    act_abl = ablate_direction(llama, direction, normalize=normalize, batch_axis=batch_axis)
    logits = act_abl(inputs)
    loss = pz.nx.nmap(lambda l, i, m: -jnp.take_along_axis(jax.nn.log_softmax(l[:-1].astype(jnp.float32), -1), i[1:, None], -1)[:, 0] * m[1:] / m[1:].sum())(
        logits.untag("seq", "vocabulary"), input_ids.untag("seq"), loss_mask.untag("seq")).sum().unwrap("batch").mean()
    return loss
lwg = jax.jit(jax.value_and_grad(get_loss), static_argnames=("normalize", "batch_axis"))

In [35]:
from micrlhf.utils.vector_storage import download_vector
import optax
import random
n_iterations = 50
batch_size = 100
max_length = 128
harmless_weight = 1
direction = download_vector("gemma-refusal-l12", overwrite=True)
optimizer = optax.adam(0.01)
opt_state = optimizer.init(direction)
for i in (bar := trange(n_iterations)):
    data_harmful = df_adv.sample(batch_size).apply(lambda x: (
        tokenizer.encode(format_prompt.format(x["goal"], "")),
        tokenizer.encode(x["target"])[1:],), axis=1).to_list()
    data_harmless = [(tokenizer.encode(format_prompt.format(prompt, "")), tokenizer.encode(completion)[1:]) for prompt, completion in [(prompts_harmless[i], completions_harmless[i]) for i in random.sample(list(range(len(prompts_harmless))), k=batch_size)]]
    # data = [x + (1,) for x in data_harmful] + [x + (harmless_weight,) for x in data_harmless]
    # data = [x + (1,) for x in data_harmful]
    data = data_harmless + data_harmful
    # data = [x + (harmless_weight,) for x in data_harmless]
    # input_ids = [x + y for x, y, _ in data]
    input_ids = [x + y for x, y in data]
    input_ids = [(x + [tokenizer.pad_token_id] * max(0, max_length - len(x)))[:max_length] for x in input_ids]
    # loss_mask = [[0] * len(x) + [z] * len(y) for x, y, z in data]
    loss_mask = [[0] * len(x) + [1] * len(y) for x, y in data]
    loss_mask = [(x + [0] * (max_length - len(x)))[:max_length] for x in loss_mask]

    input_ids = pz.nx.wrap(jnp.asarray(input_ids), "batch", "seq")
    loss_mask = pz.nx.wrap(jnp.asarray(loss_mask), "batch", "seq")

    loss, grad = lwg(direction, llama, input_ids, loss_mask)
    # direction -= 0.01 * grad
    # direction = direction / jnp.linalg.norm(direction)
    direction, opt_state = optimizer.update(grad, opt_state, direction)
    bar.set_postfix(loss=float(loss))

  0%|          | 0/50 [00:00<?, ?it/s]